In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl")
Random.seed!(2)

TaskLocalRNG()

### Generate Data

In [ ]:
function generate_Data(iterations, N, N_u, K, offdiag_sign)
    Input_Data = Dict()
    for iter in 1:iterations
        Input_Data["iter=$(iter)_Obs_Feat"] = rand(N_u);
        Input_Data["iter=$(iter)_A_true"] = rand(N,N_u);
        Input_Data["iter=$(iter)_B_true"] = generate_strictly_row_diagonally_dominant(N, 1.0,offdiag_sign);
        Input_Data["iter=$(iter)_P_dag"] = round.(rand(N, K),digits=2);
    end
    return Input_Data
end

### ETO 

In [ ]:
function Run_ETO(iterations, N, N_u, K, Input_Data)
    RST_ETO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)
        RST_ETO["iter=$(iter)_Rev"] = rev_ETO
        RST_ETO["iter=$(iter)_Price"] = price_ETO
    end
    return RST_ETO
end

### RO

In [ ]:
function Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef)
    RST_RO = Dict(); RST_RO_Two_Side = Dict()
    # psi_coef = 1
    # bd_coef = 0.000001
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        A_lb = A_true .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_true .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_true .- bd_coef .* abs.(ones(N,N));
        B_ub = B_true .+ bd_coef .* abs.(ones(N,N));

        psi_lb = 0.0 - psi_coef * 10
        psi_ub = 0.1

        obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
        rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
        println("psi_lb = ",psi_lb,",rev_RO = ",round(rev_RO,digits=6),",price_RO = ",price_RO)
        RST_RO["iter=$(iter)_Rev"] = rev_RO
        RST_RO["iter=$(iter)_Price"] = price_RO
    end
    return RST_RO
end

In [ ]:
function Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef)
    RST_RO_Two_Side = Dict()
    # phi_coef = 1
    # psi_coef = 1
    # bd_coef = 0.000001
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        A_lb = A_true .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_true .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_true .- bd_coef .* abs.(ones(N,N));
        B_ub = B_true .+ bd_coef .* abs.(ones(N,N));

        psi_lb = 0.0 - psi_coef * 10
        psi_ub = 0.1
        phi_lb = 0.0 - phi_coef * 10
        phi_ub = 0.1

        # Two-sided RO
        obj_RO_2,X_RO_2,time_RO_2 = Solve_RO(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat, P_dag,psi_lb,psi_ub,phi_lb,phi_ub)
        rev_RO_2, price_RO_2 = compute_oof(X_RO_2, A_true, B_true, Obs_Feat, P_dag)
        println("psi_lb = ",psi_lb,",phi_lb = ",phi_lb,",rev_RO = ",round(rev_RO_2,digits=6),",price_RO = ",price_RO_2)
        RST_RO_Two_Side["iter=$(iter)_Rev"] = rev_RO_2
        RST_RO_Two_Side["iter=$(iter)_Price"] = price_RO_2
    end
    return RST_RO_Two_Side
end

## offdiag is zero

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "zero"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.000001 
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
psi_coef = 1
phi_coef = 1
RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))

## offdiag is positive

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "positive"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.000001 
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
psi_coef = 1
phi_coef = 1
RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))

## offdiag is negative

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "negative"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.000001 
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
bd_coef = 0.000001 
psi_coef = 0.1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
psi_coef = 1
phi_coef = 1
RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))

## offdiag is mix

In [ ]:
Random.seed!(2)
# 设置参数
N = 5   # 产品数
K = 10   # 每个产品的选择项数量
tau = 10 # 给定常数
N_u = 2
offdiag_sign = "mix"
iterations = 10

In [ ]:
Input_Data = generate_Data(iterations, N, N_u, K, offdiag_sign);

In [ ]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);

In [ ]:
bd_coef = 0.000001 
psi_coef = 1
RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);

In [ ]:
psi_coef = 1
phi_coef = 1
RST_RO_Two_Side = Run_Two_Side_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef,phi_coef);

In [ ]:
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_RO_2 = [RST_RO_Two_Side["iter=$(iter)_Rev"] for iter in 1:iterations]

# println("Rev_ETO = ",round.(Rev_ETO,digits=4))
println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4))
println("Rev_Two_Side_Ro/Rev_ETO = ",round.(Rev_RO_2./Rev_ETO,digits=4))